In [0]:
# this only work in jupiter notebook
# pls use our default methods to read data file
# like read data from csv, textFile

dbutils.fs.put("/scenario_files/empty_header.csv","""sampleline
smapleline2
sampleline3
id,name,location
1,ravi,bangalore
2,raj,chennai
3,prasad,pune
4,mahesh,hyderabad
5,sridhar,mumbai
""",True)

Wrote 133 bytes.


True

In [0]:
df=spark.read.csv("/scenario_files/empty_header.csv",header=True)
display(df)

sampleline
smapleline2
sampleline3
id
1
2
3
4
5


In [0]:
rdd = sc.textFile("/scenario_files/empty_header.csv")
rdd.collect()

['sampleline',
 'smapleline2',
 'sampleline3',
 'id,name,location',
 '1,ravi,bangalore',
 '2,raj,chennai',
 '3,prasad,pune',
 '4,mahesh,hyderabad',
 '5,sridhar,mumbai']

In [0]:
final_rdd = rdd.zipWithIndex().filter(lambda x: x[1] > 2).map(lambda a :a[0].split(","))
final_rdd.collect()

[['id', 'name', 'location'],
 ['1', 'ravi', 'bangalore'],
 ['2', 'raj', 'chennai'],
 ['3', 'prasad', 'pune'],
 ['4', 'mahesh', 'hyderabad'],
 ['5', 'sridhar', 'mumbai']]

In [0]:
# get columns into list to create DataFrame
columns=final_rdd.collect()[0]
columns
# columns1=final_rdd.collect()[1]
# columns1
# columns2=final_rdd.collect()[2]
# columns2
# columns3=final_rdd.collect()[3]
# columns3

['3', 'prasad', 'pune']

In [0]:
# skip header/columns from data 
columns=final_rdd.first() #extract header for filter that row
data = final_rdd.filter(lambda col: col != columns)   #filter out header
data.collect()

[['1', 'ravi', 'bangalore'],
 ['2', 'raj', 'chennai'],
 ['3', 'prasad', 'pune'],
 ['4', 'mahesh', 'hyderabad'],
 ['5', 'sridhar', 'mumbai']]

In [0]:
data.toDF(columns).show() # sparkcontext

skip_rows_df = spark.createDataFrame(data = data, schema = columns) # using dataframe
skip_rows_df.display()

+---+-------+---------+
| id|   name| location|
+---+-------+---------+
|  1|   ravi|bangalore|
|  2|    raj|  chennai|
|  3| prasad|     pune|
|  4| mahesh|hyderabad|
|  5|sridhar|   mumbai|
+---+-------+---------+



id,name,location
1,ravi,bangalore
2,raj,chennai
3,prasad,pune
4,mahesh,hyderabad
5,sridhar,mumbai


In [0]:
%fs
ls /scenario_files/

path,name,size,modificationTime
dbfs:/scenario_files/empty_header.csv,empty_header.csv,133,1701843932000


In [0]:
%fs
head dbfs:/FileStore/tables/skip_line_data-1.csv

line 1
line 2
line 3
line 4
id,name,location
1,manaa,pune
2,sneha,pandarpure
3,ash,mumbai
4,rutu,sangli

In [0]:
# but if our data file in plain text format or in csv
skip_df = spark.read.format("csv")\
    .option("inferSchema", True)\
    .option("header", True)\
    .option("sep", ",")\
    .option("skipRows", 4)\
    .load("/FileStore/tables/skip_line_data-1.csv")
skip_df.display()

id,name,location
1,manaa,pune
2,sneha,pandarpure
3,ash,mumbai
4,rutu,sangli
